<a href="https://colab.research.google.com/github/EatingLupini/QuantumCircuitEvaluation/blob/main/QuantumCircuitEvaluation_Base_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
'''
Install Cirq and ProjectQ libraries
'''

!pip install cirq --quiet
!pip install projectq --quiet

In [ ]:
'''
Import Cirq and ProjectQ libraries
'''

import cirq
import numpy as np
from cirq.contrib.svg import SVGCircuit

from projectq import MainEngine
from projectq.backends import CircuitDrawer
from projectq.ops import *

In [ ]:
'''
Main Class definition
'''

class CircuitEvaluation:
  # Constructor, it defines all the necessary variables to create a circuit
  # in Cirq and ProjectQ. Some variables are used to print a better quality
  # circuit or to calculate the stats.
  def __init__(self):
    # Build circuit
    self.circuit = cirq.Circuit()
    # Get a simulator
    self.sim = cirq.Simulator()
    # Qubit map
    self.qubits = dict()
    #Gates Weight
    self.gates_weight = {
        "X": 1,
        "CNOT": 2,
        "TOFFOLI": 4,
        "FREDKIN": 4
        }
    # Qubits order
    self.qubits_order = list()
    # Result labels
    self.result_labels = list()

    # Optimization
    self.same_values = list()
    self.default_values = dict()

    # ProjectQ
    self.drawing_engine = CircuitDrawer()
    self.main_engine = MainEngine(engine_list=[self.drawing_engine])
    # ProjectQ Qubit map
    self.pq_qubits = dict()


  # Add new qubit to the circuit.
  # It adds the qubit "name" to the circuit and store it to avoid duplicates.
  def add_qubit(self, name):
    new_qubit = cirq.NamedQubit(name)
    self.qubits[name] = new_qubit
    self.qubits_order.append(new_qubit)
    self.pq_qubits[name] = self.main_engine.allocate_qubit()  # projectQ


  # Add new qubits to the circuit.
  # It calls "add_qubit" for each qubit in the list names.
  # - "same_val" needs to be a list of tuples, each tuple contains
  #   2 or more qubits that we want to be the always the same when running
  # simulations.
  # - "def_val" is a dictionary, each qubit corresponds to a constant.
  #   The specified value will be used when running simulations.
  def add_qubits(self, names, same_val, def_val):
    for name in names:
      self.add_qubit(name)
    self.same_values = same_val
    self.default_values = def_val
  

  # Add one or more gates to the circuit.
  # - "ops" is a list of gates returned by calling the bind gates functions.
  def add_ops(self, *ops):
    self.circuit.append(ops)
  

  # Finalize the circuit and add the measurement gate.
  # - "measurement_order" is an ordered list of input qubits to display.
  # - "result_map" is an ordered list of output qubits to display.
  def finalize(self, measurement_order=None, result_map=None):
    if measurement_order is not None:
      self.qubits_order = [self.qubits[m] for m in measurement_order]
    if result_map is not None:
      self.result_labels = result_map
    else:
      self.result_labels = [qb.name for qb in self.qubits_order]
    self.circuit.append([cirq.measure(self.qubits_order, key='result')])
    self.main_engine.flush()  # projectQ


  # Set gates weight
  # To elaborate the weighted depth of a circuit we need to define the weight
  # of each gate.
  # - "weights" is a dictionary, each key is a gate and
  #   corresponds to its weight.
  def set_gates_weight(self, weights):
    self.gates_weight = weights


  # Return the produced circuit.
  def get_circuit(self):
    return self.circuit
  

  # Draw the circuit.
  # Draw the circuit produced by Cirq.
  def draw(self):
    return SVGCircuit(self.circuit)
  

  # Print stat
  # Utility function to print a particular stat.
  def print_stat(self, text, val):
    nchars = 25
    stat_space = " " * (nchars - len(text))
    stat_val = str(val)
    print(text + stat_space + stat_val)


  # Print the stats of the circuit.
  # - number of qubits
  # - number of gates
  # - number of gates per type
  # - depth of the circuit
  # - weighted depth of the circuit
  def print_stats(self):
    print("STATS")

    # Number of qubits
    self.print_stat("Number of qubits:", len(self.qubits))

    # Number of gates
    num_gates_all = 0
    num_gates = dict()
    for moment in self.circuit.moments:
      for op in moment.operations:
        if "MeasurementGate" not in str(op.gate):
          num_gates_all += 1
          if op.gate in num_gates:
            num_gates[op.gate] += 1
          else:
            num_gates[op.gate] = 1
    self.print_stat("Number of gates (All):", num_gates_all)

    # Number of gates per type
    for gate in num_gates.keys():
      self.print_stat(f"  {gate}:", num_gates[gate])

    # Depth (len - 1 because of the measurement gate)
    self.print_stat("Circuit depth:", len(self.circuit.moments) - 1)

    # Depth with weights
    depth_full = 0
    for moment in self.circuit.moments:
      max_w = 1
      all_measurements = True
      for op in moment.operations:
        if str(op.gate) in self.gates_weight:
          max_w = max(max_w, self.gates_weight[str(op.gate)])
          all_measurements = False
      if all_measurements:
        max_w = 0
      depth_full += max_w
    self.print_stat("Circuit weighted depth:", depth_full)
  

  # Beautify results.
  # Utility function to display the results of simulations.
  def print_results(self, full_data):
    qbs_len = list()
    res_len = list()

    txt_cols = "|"
    for qb in self.qubits_order:
      qbs_len.append(len(qb.name))
      txt_cols += qb.name + "|"
    txt_cols += "  ->  |"
    for name in self.result_labels:
      res_len.append(len(name))
      txt_cols += name + "|"
    
    print("." + "_" * (len(txt_cols) - 2) + ".")
    print(txt_cols)

    txt_sep = "|"
    for qbl in qbs_len:
      txt_sep += "=" * qbl + "|"
    txt_sep += "  ->  |"
    for rl in res_len:
      txt_sep += "=" * rl + "|"

    print(txt_sep)

    for data in full_data:
      qbs, res = data
      txt_row = "|"
      for i in range(len(qbs)):
        qb, qbl = str(qbs[i]), qbs_len[i]
        txt_row += qb + " " * (qbl - len(qb)) + "|"
      txt_row += "  ->  |"
      for i in range(len(res)):
        rv, rl = str(res[i]), res_len[i]
        txt_row += rv + " " * (rl - len(rv)) + "|"
      print(txt_row)
    print("*" + "=" * (len(txt_cols) - 2) + "*")


  # Run a simulation and return the output values.
  # Draws results by default.
  def simulate(self, qb_state, draw=True):
    qb_state_dec = 0
    for b in qb_state:
        qb_state_dec = 2 * qb_state_dec + b
    result = self.sim.simulate(self.circuit,
                          qubit_order=self.qubits_order,
                          initial_state=qb_state_dec)
    if draw:
      self.print_results([(qb_state, result.measurements['result'])])
    return result


  # Run a simulation for each input.
  # Return a complete list of pairs input-output values
  def simulate_all(self, draw=True):
    full_data = list()
    num_qubits = len(self.qubits_order)
    num_comb = 2 ** num_qubits
    for current_val in range(num_comb):
      temp_str_state = format(current_val, f'0{num_qubits}b')
      temp_qb_state = [int(c) for c in temp_str_state]
      
      # check
      skip = False

      # check default values
      for i in range(num_qubits):
        cur_qb_name = self.qubits_order[i].name
        cur_qb_value = temp_qb_state[i]
        if cur_qb_name not in self.default_values or \
          cur_qb_value == self.default_values[cur_qb_name]:
          continue
        skip = True
      
      # check same values
      if not skip:
        sv = dict()
        for i in range(num_qubits):
          cur_qb_name = self.qubits_order[i].name
          cur_qb_value = temp_qb_state[i]
          for same in self.same_values:
            if cur_qb_name in same:
              if same in sv:
                if cur_qb_value != sv[same]:
                  skip = True
                  break
              else:
                sv[same] = cur_qb_value
          if skip:
            break

      if skip:
        continue
      
      res = self.simulate(temp_qb_state, False)
      full_data.append((temp_qb_state, res.measurements['result']))
    if draw:
      self.print_results(full_data)
    return full_data
  

  # Print latex
  # Print the latex code to later compile. It also adds labels to the circuit.
  def print_latex(self, lst_in, lst_out):
    latex_txt = self.drawing_engine.get_latex()
    lines = latex_txt.splitlines()
    
    max_x = 0
    for line_index in range(len(lines)):
      line = lines[line_index]
      if '\\node' in line:
        cur_x = float(line[line.rfind("(")+1:line.rfind(",")])
        if cur_x > max_x:
          max_x = cur_x
      if '\\node[none]' in line:
        num_wire = int(line[line.rfind("-")+1:line.rfind(")")])
        check_valid = line[line.rfind("{")+1:line.rfind("}")]
        if check_valid:
          txt_cur = line[line.rfind("{")-5:line.rfind("}")]
          txt_new = "$\\Ket{" + lst_in[num_wire] + "}$"
          lines[line_index] = line.replace(txt_cur, txt_new)
    
    for i in range(len(lst_out)):
      wire = lst_out[i]
      si = str(i)
      smx = str(max_x + 1)
      lines.insert(len(lines)-2,
          "\\node[none] (line" + si + "_gate999) at (" + smx + ",-" + si + ") {$\\Ket{" + wire + "}$};")
      lines.insert(len(lines)-2,
          "\\draw (line" + si + "_gate0) edge[edgestyle] (line" + si + "_gate999);")
    
    latex_edited = ""
    for line in lines:
      latex_edited += line + "\n"
    
    print(latex_edited)


  # Bind gates.
  # It calls the right gate function for both Cirq and ProjectQ.
  def X(self, *qbs):
    X | self.pq_qubits[qbs[0]]
    return cirq.X(self.qubits[qbs[0]])
  def CNOT(self, *qbs):
    CNOT | (self.pq_qubits[qbs[0]], self.pq_qubits[qbs[1]])
    return cirq.CNOT(self.qubits[qbs[0]], self.qubits[qbs[1]])
  def TOFFOLI(self, *qbs):
    Toffoli | (self.pq_qubits[qbs[0]], self.pq_qubits[qbs[1]], self.pq_qubits[qbs[2]])
    return cirq.TOFFOLI(self.qubits[qbs[0]], self.qubits[qbs[1]], self.qubits[qbs[2]])
  def FREDKIN(self, *qbs):
    C(Swap) | (self.pq_qubits[qbs[0]], self.pq_qubits[qbs[1]], self.pq_qubits[qbs[2]])
    return cirq.FREDKIN(self.qubits[qbs[0]], self.qubits[qbs[1]], self.qubits[qbs[2]])
